<a href="https://colab.research.google.com/github/Soufianelotfi-lab/Classification_Formes_Ondes_6G/blob/main/Models1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**`I. PREPARATION DE DATA POUR LE MODELE :`**


**`1. Préparation de l’environnement et des librairies`**


In [ ]:
!pip install scipy tqdm -q
!pip install torch torchvision torchaudio -q

import numpy as np
import scipy.io as sio
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device utilisé :", device)

**`2. Upload des dataset`**

- Cette cellule permet d’accéder au dataset stocké sur Google Drive et de le rendre disponible dans l’environnement Google Colab.

In [ ]:
from google.colab import drive, files

drive.mount('/content/drive')

# Adapter le chemin si le fichier est stocké ailleurs dans le Drive
files.download("/content/drive/MyDrive/5ème/Projet_techno/data_set_40k.mat")


**`3. Chargement des fichiers MATLAB (.mat)`**

- Cette cellule permet de charger le fichier de données MATLAB (.mat) stocké sur Google Drive.
Le chemin du fichier est défini puis le contenu est importé en mémoire à l’aide de scipy.io.loadmat, ce qui rend les signaux accessibles en Python sous forme de dictionnaire

In [ ]:
# Chemin vers le fichier de données MATLAB
signals_path = "/content/drive/MyDrive/5ème/Projet_techno/data_set_40k.mat"

print("Chemin signaux :", signals_path)
mat_signals = sio.loadmat(signals_path)
print("Fichier des signaux chargé.")



**`4. Exploration du contenu des fichiers .mat`**

- Pour savoir le nom du fichier qui est telechargé (ici on trouve que c'est data_set) apres globals.

In [ ]:
print("Variables dans le fichier des signaux :")
print(mat_signals.keys())

**`5. Extraction brute de X et y depuis les fichiers MATLAB`**

In [ ]:
X = mat_signals["data_set"]
print("Shape brute X :", X.shape)


**`5.a. Tracer un signal`**

- Cette cellule permet de visualiser un signal brut extrait du dataset afin d’avoir un aperçu temporel des données.

In [ ]:
import matplotlib.pyplot as plt

# Sélection de l'indice du signal à visualiser
index = 20000

plt.figure(figsize=(12,4))
plt.plot(X[index])
plt.title(f"Signal numéro {index}")
plt.xlabel("Échantillons")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()


**`5.b. Tracer des échantillons: `**

- Cette cellule permet de visualiser les échantillons discrets d’un signal sélectionné dans le dataset.

In [ ]:
import matplotlib.pyplot as plt
# Sélection du signal et de la plage d'échantillons
index = 33000 # Indice du signal sélectionné dans le dataset (0 à 9999 : classe 1, 10000 à 19999 : classe 2, 20000 à 29999 : classe 3, 30000 à 39999 : classe 4)
start = 0 # Indice de début des échantillons à afficher
end = 4096  # Indice de fin des échantillons à afficher

plt.figure(figsize=(12,4))
plt.stem(X[index][start:end])
plt.title("Classe 4 ")
plt.xlabel("Index des échantillons")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()


**`6. Inspection des dimensions de X`**

- Cette cellule permet de vérifier le type et les dimensions du tableau contenant les données.
Cela garantit que la structure de X est conforme avant de passer aux étapes de traitement ou d’apprentissage.

In [ ]:

print("Type X :", type(X))
print("Shape X :", np.shape(X))



**`6. Distribution des échantillons (avant reshape)`**



- Cette cellule permet d’analyser la distribution globale des valeurs I/Q avant toute mise en forme des données. Les échantillons sont aplatis afin d’observer la répartition statistique des amplitudes sur l’ensemble du dataset. L’analyse montre que les valeurs sont majoritairement concentrées autour de zéro, sans présence de valeurs extrêmes significatives, ce qui confirme une distribution stable avant le reshape et l’apprentissage.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.hist(X.flatten(), bins=200)
plt.title("Histogramme des valeurs avant reshape")
plt.xlabel("Valeurs")
plt.ylabel("Fréquence")
plt.show()


**`7. Changement de la structure de X : `**


*   La structure initiale de X n’est pas directement compatible avec PyTorch, qui attend des données sous la forme (nombre de signaux, nombre de canaux, nombre d’échantillons).
Les valeurs I et Q étant concaténées dans une seule dimension, la matrice est reformée afin de séparer explicitement les deux canaux.
*   Pour en faire, nous avons restruture le formt de X :



In [ ]:
import numpy as np
# Affichage de la forme initiale des données
print("Shape X avant traitement :", X.shape)   # (40000, 4096)

# Nombre total de signaux
N = X.shape[0]   # 40000 signaux

# Nombre total de valeurs par signal (I + Q concaténés)
total_features = X.shape[1]  # 4096 valeurs par signal

# Vérification que la dimension est divisible par 2 (I et Q)
if total_features % 2 != 0:
    raise ValueError("Le nombre de features par signal n'est pas divisible par 2, ce n'est pas cohérent avec 2 canaux I/Q.")

# Nombre d'échantillons par canal
L = total_features // 2   # 2048 échantillons par canal
print("Nombre de canaux :", 2)
print("Longueur par canal :", L)

# On reforme X en (N, 2, L) = (40000, 2, 2048)
X = X.reshape(N, 2, L)
print("le Shape X après reshape :", X.shape)


**- FFT**

- Cette cellule permet de transformer les signaux du domaine temporel vers le domaine fréquentiel en calculant la FFT et sa magnitude, tout en conservant la structure des données.
Elle est à exécuter uniquement si l’analyse ou l’apprentissage est réalisé dans le domaine fréquentiel.
Dans le cas contraire, si l’on travaille directement dans le domaine temporel, cette cellule peut être ignorée.

In [ ]:
import torch
import torch.fft
import numpy as np

print("Shape original X:", X.shape)


# 0) Si X est un numpy ndarray, convertir en Tensor
if isinstance(X, np.ndarray):
    X = torch.tensor(X, dtype=torch.float32)
    print("X converti en Tensor :", X.shape)

# 1) Calcul FFT magnitude
def compute_fft_keep_shape(X):
    """
    X : (batch, 2, L)
    Retour : (batch, 1, L)
    """
    I = X[:, 0, :]              # (batch, L)
    Q = X[:, 1, :]              # (batch, L)

    # Signal complexe
    x_complex = I + 1j * Q      # complexe = compatible pytorch

    # FFT
    X_fft = torch.fft.fft(x_complex)

    # Magnitude
    X_mag = torch.abs(X_fft)    # (batch, L)

    # Normalisation par signal
    X_mag = X_mag / (X_mag.max(dim=1, keepdim=True)[0] + 1e-6)

    # CNN 1D attend (batch, channels, length)
    X_mag = X_mag.unsqueeze(1)  # (batch, 1, L)

    return X_mag.float()

X = compute_fft_keep_shape(X)

print("Shape X après FFT :", X.shape)


In [ ]:
import matplotlib.pyplot as plt

# Choisir l'indice du signal
index = 3001

# Extraire le spectre fréquentiel (magnitude FFT)
fft_signal = X[index, 0].cpu().numpy()

# Affichage
plt.figure(figsize=(10,4))
plt.plot(fft_signal)
plt.xlabel("Indice fréquentiel")
plt.ylabel("Amplitude normalisée")
plt.title(f"Signal {index} après FFT (domaine fréquentiel)")
plt.grid(True)
plt.show()


**`8. Distribution des échantillons I/Q (apres reshape)`**




- Cette cellule permet de visualiser la distribution des composantes I et Q lorsque l’on travaille dans le domaine temporel.
Les échantillons des deux canaux sont séparés puis représentés sous forme d’histogrammes afin d’analyser leur répartition statistique.
Cette visualisation permet de comparer le comportement des canaux I et Q avant toute transformation fréquentielle.

In [ ]:
# À utiliser uniquement si l'analyse est réalisée dans le domaine temporel
I = X[:,0,:].flatten()
Q = X[:,1,:].flatten()

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.hist(I, bins=200)
plt.title("Histogramme du canal I")

plt.subplot(1,2,2)
plt.hist(Q, bins=200)
plt.title("Histogramme du canal Q")

plt.show()


**`Tracer des echantillons de I et Q :`**

- Dans cette étape, les échantillons des composantes I et Q d’un même signal sont tracés séparément dans le domaine temporel.

In [ ]:
import matplotlib.pyplot as plt

index = 0     # choisis le signal que vous voulez tracer
start = 0     # début
end = 4096     # fin (par exemple 200 premiers échantillons)

I = X[index, 0, start:end]   # canal I
Q = X[index, 1, start:end]   # canal Q

plt.figure(figsize=(14,5))

# Signal I
plt.subplot(1,2,1)
plt.stem(I)
plt.title(f"Canal I — échantillons {start}:{end}")
plt.xlabel("Index")
plt.ylabel("Amplitude")
plt.grid(True)

# Signal Q
plt.subplot(1,2,2)
plt.stem(Q)
plt.title(f"Canal Q — échantillons {start}:{end}")
plt.xlabel("Index")
plt.ylabel("Amplitude")
plt.grid(True)

plt.show()


**`8. Creation des labels :`**

- Dans cette étape, les labels ne sont pas directement extraits du fichier de données mais générés manuellement.
Un vecteur de labels est créé en supposant une répartition équilibrée des signaux, avec le même nombre d’échantillons pour chaque classe.


In [ ]:
n_classes = 4
n_per_class = 10000
Y = np.repeat(np.arange(n_classes), n_per_class)

print("Premiers labels :", Y[:20]) #Pour afficher les 20 premiers labels pour vérifier le début du vecteur
print("Derniers labels :", Y[-20000:]) #Pour afficher les 20000 derniers labels pour vérifier la fin du vecteur
print("Shape y :", Y.shape)
print("Valeurs uniques :", np.unique(Y))

In [ ]:
print("min =", X.min())
print("max =", X.max())
print("mean =", X.mean())
print("std =", X.std())


**`9. Creation du Dataset:`**     
  - La classe WaveformDataset permet non seulement de structurer les 40000 signaux I/Q dans un format compatible avec PyTorch, mais aussi d’associer automatiquement chaque signal à son label grâce à la méthode __getitem__, qui renvoie pour un indice donné le couple (signal_i, label_i).



In [ ]:
class WaveformDataset(Dataset):
    def __init__(self, X, Y):
        # X : numpy (N, 2, 2304)
        # Y : numpy (N,)
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.long)

    def __len__(self):
        # Nombre total de signaux
        return len(self.X)

    def __getitem__(self, idx):
        # Un signal+son label
        return self.X[idx], self.Y[idx]

# Création du dataset complet
dataset = WaveformDataset(X, Y) # juste pour s'assurer que la classe fait son travail, fonctionne correctement sinon c'est facultatif (c'est pas le data qu'on va utiliser)
print("Taille du dataset :", len(dataset))



**`10. Division de Dataset pour un partie d'entrainemenet et une partie de test`**.










In [ ]:
# Première division entrainement + test
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,     # reproductibilité
    stratify=Y,          # même proportion de chaque classe, que ca soit dns la partie d'entrainement ou test, les 4 classes doivent etre équilbré de la meme quantité
    shuffle=True         # mélanger les signaux
)

print("Taille X_train :", X_train.shape)
print("Taille X_test  :", X_test.shape)
print("Taille Y_train :", Y_train.shape)
print("Taille Y_test  :", Y_test.shape)

In [ ]:
# Deuxième division: créer un ensemble de validation à partir de l'ensemble d'entraînement (validation + entrainement)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train,
    Y_train,
    test_size=0.125,
    random_state=42,
    stratify=Y_train,
    shuffle=True
)

print("Taille X_train :", X_train.shape)
print("Taille X_val   :", X_val.shape)
print("Taille X_test  :", X_test.shape)
print("Taille Y_train :", Y_train.shape)
print("Taille Y_val   :", Y_val.shape)
print("Taille Y_test  :", Y_test.shape)


**`- Sassurer que la division est equlibrée`**

In [ ]:
print("Train counts:", np.unique(Y_train, return_counts=True))
print("Val counts  :", np.unique(Y_val, return_counts=True))

- Cette cellule permet de vérifier que les classes sont équilibrées dans chaque ensemble de données (entraînement, validation et test).
- La distribution des labels est représentée sous forme d’histogrammes afin de s’assurer que chaque classe est correctement représentée après le découpage du dataset.
- Cette vérification permet d’éviter tout biais lié à un déséquilibre des classes lors de l’apprentissage et de l’évaluation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_distribution(y, title):
    classes, counts = np.unique(y, return_counts=True)

    plt.figure(figsize=(6,4))
    plt.bar(classes, counts)
    plt.title(title)
    plt.xlabel("Classes")
    plt.ylabel("Nombre d'exemples")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.show()

plot_distribution(Y_train, "Distribution Train")
plot_distribution(Y_val, "Distribution Validation")
plot_distribution(Y_test, "Distribution Test")


**`- Normalisation des donnés (facultatif)`**

- Cette cellule permet de normaliser les données à partir des statistiques de l’ensemble d’entraînement.
- La moyenne et l’écart-type sont calculés puis utilisés pour normaliser les ensembles d’entraînement, de validation et de test de manière cohérente.
- Cette étape est facultative, mais elle peut améliorer la stabilité et la convergence de l’apprentissage.

In [ ]:
mean = X_train.mean(axis=(0, 2), keepdims=True)   # shape (1, 2, 1)
std  = X_train.std(axis=(0, 2), keepdims=True) + 1e-8

X_train_norm = (X_train - mean) / std
X_val_norm   = (X_val   - mean) / std
X_test_norm  = (X_test  - mean) / std

**`11. Creation de la data sous forme (signaux+labls):`**






*   Dans cette étape, les données sont regroupées sous la forme (signal, label) à l’aide de la classe WaveformDataset qu'on a crée avant.
- Les ensembles d’entraînement, de validation et de test sont ainsi préparés à partir des données qu'on vient de séparer.
- Cette structuration permet une utilisation directe avec les outils PyTorch (DataLoader, entraînement, évaluation).






In [ ]:
train_dataset = WaveformDataset(X_train_norm, Y_train)
val_dataset   = WaveformDataset(X_val_norm,   Y_val)
test_dataset  = WaveformDataset(X_test_norm,  Y_test)

print("Taille train_dataset :", len(train_dataset))
print("Taille val_dataset   :", len(val_dataset))
print("Taille test_dataset  :", len(test_dataset))

**`12. Dataloader :`**
- Nous avons utilisé le DataLoader de PyTorch pour organiser les données en mini-lots (batch), ce qui permet d’envoyer les signaux au modèle par petites séquences durant l’entraînement. trois DataLoaders ont été créés : train_loader et val_loader pour entraîner le modèle et test_loader pour évaluer ses performances.

In [ ]:
batch_size = 32
#La partie d'entrainement
train_loader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle=True    # mélange les échantillons à chaque epoch pour améliorer la généralisation et éviter que le modèle apprenne l'ordre des données et réduire le risque d'overfitting.
)

# Loader de validation
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False  # pas besoin de mélanger ici
)

#La partie de test
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False   # éviter que le modèle apprenne l'ordre des données
)

# Petit test pour vérifier qu'un batch ressemble à ce qu'on veut (optionnel juste pour vérification)
for X_batch, Y_batch in train_loader:
    print("Shape X_batch :", X_batch.shape)  # attendu : (32, 2, 2304)
    print("Shape y_batch :", Y_batch.shape)  # attendu : (32,)
    print("Quelques labels :", Y_batch[:10])
    break


**`II. CREATION DE MODELE`**

**`1. Définiton du modèle :`**

- Les deux premiers modèles sont conçus pour un traitement dans le domaine temporel.
- Le premier modèle, plus profond et comportant un grand nombre de couches, offre une forte capacité de représentation mais présente un surapprentissage.
- Le second modèle constitue une version optimisée et plus compacte, permettant
de limiter un peu ce surapprentissage (mais qui persiste toujours).
- Enfin, le troisième modèle est utilisé lorsque l’analyse est réalisée dans le domaine fréquentiel, après transformation des signaux par FFT.




**`1.1 Models pour D.temporel`**

**Modele 1**

In [ ]:
# Définition de la classe du modèle de Deep Learning
class WaveformClassifier_3(nn.Module):

    # Constructeur du modèle
    def __init__(self):
        # Appel du constructeur de la classe parente nn.Module
        super(WaveformClassifier_3, self).__init__()

        # Définition du bloc CNN pour l'extraction de caractéristiques locales
        self.cnn = nn.Sequential(
            # Première convolution : passage de 2 canaux (I/Q) à 32 filtres
            nn.Conv1d(2, 32, kernel_size=9, padding=4),
            # Normalisation pour stabiliser l'entraînement
            nn.BatchNorm1d(32),
            # Fonction d'activation non linéaire
            nn.ReLU(),

            # Deuxième convolution : augmentation du nombre de filtres
            nn.Conv1d(32, 64, kernel_size=7, padding=3),
            # Normalisation
            nn.BatchNorm1d(64),
            # Activation
            nn.ReLU(),

            # Troisième convolution : représentation plus abstraite
            nn.Conv1d(64, 128, kernel_size=5, padding=2),
            # Normalisation
            nn.BatchNorm1d(128),
            # Activation
            nn.ReLU(),

            # Réduction de la dimension temporelle
            nn.MaxPool1d(2)
            # Dropout possible pour réduire le surapprentissage (désactivé ici)
            # nn.Dropout(0.2)
        )

        # Définition du LSTM pour modéliser les dépendances temporelles
        self.lstm = nn.LSTM(
            # Nombre de features en entrée du LSTM (sortie du CNN)
            input_size=128,
            # Taille de l'état caché
            hidden_size=32,
            # Nombre de couches LSTM
            num_layers=1,
            # LSTM bidirectionnel (avant + arrière)
            bidirectional=True
            # Dropout possible entre couches LSTM (désactivé ici)
            # dropout=0.3
        )

        # Définition de la couche fully connected pour la classification finale
        self.fc = nn.Linear(32 * 2, 4 ) # 32*2 car LSTM bidirectionnel (dans 32 (Taille de l'état caché) qui represente la sortie du LSTM)

    # Définition du passage avant (forward)
    def forward(self, x):
        # x est de forme (batch_size, 2, 2304)

        # Passage des données dans le CNN
        x = self.cnn(x)               # (batch_size, 128, L')

        # Transposition pour correspondre au format attendu par le LSTM
        x = x.transpose(1, 2)         # (batch_size, L', 128)

        # Passage dans le LSTM
        lstm_out, _ = self.lstm(x)    # (batch_size, L', 64)

        # Agrégation temporelle par moyenne sur l'axe du temps
        last_time_step = lstm_out.mean(dim=1)

        # Passage dans la couche dense pour obtenir les logits
        out = self.fc(last_time_step)

        # Retour de la sortie du modèle
        return out


**- Optimisation de l’Architecture du Modèle**

In [ ]:
class WaveformClassifier_2(nn.Module):
    def __init__(self):
        super(WaveformClassifier_2, self).__init__()
        # La couche CNN
        self.cnn = nn.Sequential(
            nn.Conv1d(2, 32, kernel_size=9, padding=4),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Conv1d(32, 48, kernel_size=7, padding=3),
            nn.BatchNorm1d(48),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.3)  # nouveau : 30% des neurones désactivés aléatoirement
        )
        #La couche LSTM
        self.lstm = nn.LSTM(
            input_size= 48,
            hidden_size= 32, #changr 64 par 32
            num_layers=1, #j'ai ajouté une couche -----------------------------------------------------------------------------------------------
            batch_first=True, #le batch est en premier dans les dimensions : (batch, seq, features)
            bidirectional= True, # LSTM dans les deux sens (avant + arrière)
            #dropout=0.3
        )
        #La couche dense
        self.fc = nn.Linear(32*2 , 4)  # 64*2 car bidirectionnel,128 sorties, 4 classes à prédire
        #self.fc = nn.Linear(32, 4)  # 64*2 car bidirectionnel


    def forward(self, x):
        # x : (batch_size, 2, 2304)
        x = self.cnn(x)             # (batch, 64, L)
        x = x.transpose(1, 2)       #  (batch, L, 64)

        lstm_out, _ = self.lstm(x)  # (batch, L, 128)
        #last_time_step = lstm_out[:, -1, :]  # (batch, 128)
        last_time_step = lstm_out.mean(dim=1)

        out = self.fc(last_time_step)
        return out

**`1.2 Model pour D.fréquentiel`**

In [ ]:
class WaveformClassifier_DF(nn.Module):
    def __init__(self):
        super(WaveformClassifier_DF, self).__init__()
        # La couche CNN
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=9, padding=4), # un seul canal d'entrée
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=7, padding=3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64,128, kernel_size=5, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.3)  # nouveau : 30% des neurones désactivés aléatoirement

        )
        #La couche LSTM
        self.lstm = nn.LSTM(
            input_size= 128,
            hidden_size= 64,
            num_layers=2, # Le nombre de couche LSTM
            batch_first=True, #le batch est en premier dans les dimensions : (batch, seq, features)
            bidirectional= True, # LSTM dans les deux sens (avant + arrière)
            dropout= 0.3 # si nous travaillons avec plus d'une seule couche
        )
        #La couche dense
        self.fc = nn.Linear(64 * 2, 4)  #  *2 car bidirectionnel, 4 classes à prédire
        #self.fc = nn.Linear(32, 4)  # 64*2 car bidirectionnel


    def forward(self, x):
        x = self.cnn(x)
        x = x.transpose(1, 2)

        lstm_out, _ = self.lstm(x)
        last_time_step = lstm_out.mean(dim=1)

        out = self.fc(last_time_step)
        return out

**`2. Créer le modèle + choisir le device`**


* Cette instruction détecte automatiquement si un GPU (CUDA) est disponible et choisit le meilleur matériel pour exécuter le modèle, sinon elle utilise le CPU. Ensuite, le modèle WaveformClassifier est créé et transféré sur ce device afin d’accélérer l’entraînement et garantir que toutes les opérations se font sur le même support.




In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device utilisé :", device)

model = WaveformClassifier_DF().to(device) #mettre le model voulu


**`- Initialisation des poids`**

- Cette cellule définit une initialisation personnalisée des poids du modèle afin d’améliorer la stabilité et la convergence de l’entraînement.
- Des méthodes adaptées sont utilisées selon le type de couche (CNN, couche dense ou LSTM).
- Cette étape permet de partir d’un état initial plus favorable que l’initialisation par défaut.

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)


**`3. Définir la loss et l’optimiseur`**

- Cette étape applique l’initialisation des poids définie précédemment à l’ensemble du modèle.
- La fonction de perte et l’optimiseur sont ensuite configurés afin d’entraîner le réseau pour notre problème de classification multi-classes.


In [ ]:
# Application de l'initialisation personnalisée des poids au modèle
model.apply(init_weights)

# Définition de la fonction de perte pour une classification à 4 classes
criterion = nn.CrossEntropyLoss()                 # pour la classification en 4 classes

# Définition de l'optimiseur Adam pour l'entraînement du modèle
optimizer = torch.optim.Adam(model.parameters(),  # tous les poids du modèle
                             lr=0.0001 # Taux d'apprentissage
                             )


**`4. Fonction d'entraînement`**

- Cette fonction réalise une époque complète d’entraînement du modèle.
Pour chaque batch, elle effectue la propagation avant, le calcul de la perte, la rétropropagation et la mise à jour des poids.
Elle retourne ensuite la perte moyenne et la précision sur l’ensemble de l’époque.

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
   # Parcours des batches de données
    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # 1) Reset gradients
        optimizer.zero_grad()

        # 2) Propagation avant (Forward)
        outputs = model(X_batch)

        # 3) Calcul de la fonction de perte (loss)
        loss = criterion(outputs, y_batch)

        # 4) Rétropropagation (Backprop)
        loss.backward()

        # 5) Update poids
        optimizer.step()

        # Stats
        running_loss += loss.item() * X_batch.size(0)
        _, preds = torch.max(outputs, 1) # il permt de detrminer les classes pédites dans tout le batch en se basant sur le maximum
        correct += (preds == y_batch).sum().item() #il compare les predictions au vraies classes
        total += y_batch.size(0)

    return running_loss / total, correct / total


**`5. Fonction d'evaluation`**

In [ ]:
def eval_one_epoch(model, loader, criterion, device):
    model.eval()  # pas d’apprentissage
    running_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # pas de calcul de gradient
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            running_loss += loss.item() * X_batch.size(0)

            _, preds = torch.max(outputs, 1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    return running_loss / total, correct / total


**`6. Fonction test / validation`**

In [ ]:
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            running_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    return running_loss / total, correct / total


**`III. Apprentissage et evaluation du modèle :`**

**`7. La boucle d'apprentissage`**

In [ ]:
train_losses = []
val_losses = []
train_accuracy = []
val_accuracy = []

num_epochs = 50
best_val_loss = float('inf')   # pour suivre le meilleur modèle

for epoch in range(num_epochs):
    # Entraînement pour une epoch
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)

    # Validation
    val_loss, val_acc = eval_one_epoch(model, val_loader, criterion, device)

    # Sauvegarde des courbes
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracy.append(train_acc)
    val_accuracy.append(val_acc)

    # Affichage
    print(f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")

    # Sauvegarde automatique du meilleur modèle
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print(">>> Meilleur modèle sauvegardé (val_loss améliorée)")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_loss_curves(train_losses, val_losses):
    epochs = range(1, len(train_losses) + 1)

    # Trouver le meilleur epoch (min de la val_loss)
    best_epoch = np.argmin(val_losses) + 1
    best_val = val_losses[best_epoch - 1]

    plt.figure()
    plt.plot(epochs, train_losses, label="Train Loss")
    plt.plot(epochs, val_losses, label="Validation Loss")

    # Point rouge du meilleur epoch
    plt.scatter(best_epoch, best_val, color='red', s=60)

    # Ligne verticale
    plt.axvline(best_epoch, color='red', linestyle='--', alpha=0.7,
                label=f"Best Epoch = {best_epoch}")

    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Courbe de Loss (Train vs Validation)")
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_accuracy_curves(train_accuracies, val_accuracies):
    epochs = range(1, len(train_accuracies) + 1)

    plt.figure()
    plt.plot(epochs, [a * 100 for a in train_accuracies], label="Train Acc (%)")
    plt.plot(epochs, [a * 100 for a in val_accuracies], label="Val Acc (%)")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.title("Courbe d’Accuracy (Train vs Validation)")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
plot_loss_curves(train_losses, val_losses)
plot_accuracy_curves(train_accuracy, val_accuracy)


In [ ]:
model.load_state_dict(torch.load("best_model.pth"))
model.eval()


**`8. Phase de test`**

In [ ]:
from IPython.display import display, HTML

test_loss, test_acc = evaluate(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc*100:.2f}%")

html_table = f"""
<table border="1" style="
    border-collapse: collapse;
    text-align: center;
    width: 60%;
    font-size: 20px;
    margin-top: 20px;
">
    <tr style="background-color: #f0f0f0; font-weight: bold;">
        <th style="padding: 10px;">Metric</th>
        <th style="padding: 10px;">Value</th>
    </tr>
    <tr>
        <td style="padding: 12px;">Test Loss</td>
        <td style="padding: 12px;">{test_loss:.4f}</td>
    </tr>
    <tr>
        <td style="padding: 12px;">Test Accuracy</td>
        <td style="padding: 12px;">{test_acc*100:.2f}%</td>
    </tr>
</table>
"""

display(HTML(html_table))


**`9. Matrice de confusion`**

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

model.eval()

y_true_list = []
y_pred_list = []

# 1) On parcourt tout le test_loader
with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)

        outputs = model(X)
        _, predicted = outputs.max(1)

        # On stocke labels réels et prédits
        y_true_list.extend(y.cpu().numpy())
        y_pred_list.extend(predicted.cpu().numpy())

# 2) Calcul de la matrice de confusion
cm = confusion_matrix(y_true_list, y_pred_list)

print("Matrice de confusion :")
print(cm)

# 3) Affichage graphique
plt.figure(figsize=(6, 6))
plt.imshow(cm, cmap="Greens")
plt.title("Matrice de Confusion")
plt.xlabel("Prédictions")
plt.ylabel("Vérités")
plt.colorbar()

# Afficher les valeurs dans les cases
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center", color="black")

plt.show()


In [ ]:
# ===== TABLEAU PRECISION / RECALL / F1 EN % =====

import torch
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:   # ou val_loader
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ---- Métriques par classe (en %) ----
precision_pc = precision_score(all_labels, all_preds, average=None) * 100
recall_pc    = recall_score(all_labels, all_preds, average=None) * 100
f1_pc        = f1_score(all_labels, all_preds, average=None) * 100
support_pc   = pd.Series(all_labels).value_counts().sort_index().values

# ---- Tableau ----
df_metrics = pd.DataFrame({
    "Precision (%)": precision_pc,
    "Recall (%)": recall_pc,
    "F1-score (%)": f1_pc,
    "Support": support_pc
})

# ---- Moyennes ----
df_metrics.loc["Macro avg"] = [
    precision_score(all_labels, all_preds, average="macro") * 100,
    recall_score(all_labels, all_preds, average="macro") * 100,
    f1_score(all_labels, all_preds, average="macro") * 100,
    sum(support_pc)
]

df_metrics.loc["Weighted avg"] = [
    precision_score(all_labels, all_preds, average="weighted") * 100,
    recall_score(all_labels, all_preds, average="weighted") * 100,
    f1_score(all_labels, all_preds, average="weighted") * 100,
    sum(support_pc)
]

# ---- Arrondi pour affichage ----
df_metrics = df_metrics.round(2)

df_metrics
